Training part

In [3]:
from __future__ import division
import matplotlib.pyplot as plt
import numpy as np
import dynet as dy

In [8]:
#def similarity():
    
#data is already in data numpy matrix

class decomposable_attention():
    def __init__(self,embedding_dim,debug=False):
        self.pc = dy.Model()
        self.debug = debug
        self.embedding_dim = embedding_dim 
        self.embedding_matrix = self.pc.add_lookup_parameters((16800,embedding_dim))
        self.data1 = np.random.randint(16800,size=(10000,50))
        self.data2 = np.random.randint(16800,size=(10000,50))
        self.len1 = 1+np.random.randint(50,size=10000)
        self.len2 = 1+np.random.randint(50,size=10000)
        self.labels = np.random.randint(2,size=10000)
        
        self.w2 = self.pc.add_parameters((embedding_dim,2*embedding_dim))
        self.b2 = self.pc.add_parameters((embedding_dim,1))
        
        self.w3 = self.pc.add_parameters((embedding_dim,2*embedding_dim))
        self.b3 = self.pc.add_parameters((embedding_dim,1))
        
        self.w = self.pc.add_parameters((1,2*embedding_dim))
        self.b = self.pc.add_parameters((1,1))
        
        
    def forward(self,x1,x2,label,k,tl):
        
        debug = self.debug
        len1 = self.len1
        len2 = self.len2
        embedding_dim = self.embedding_dim
        
        w = dy.parameter(self.w)
        b = dy.parameter(self.b)
        w2 = dy.parameter(self.w2)
        b2 = dy.parameter(self.b2)
        w3 = dy.parameter(self.w3)
        b3 = dy.parameter(self.b3)
        
        embeds1 = dy.reshape(dy.lookup_batch(self.embedding_matrix,x1),(len1[k],embedding_dim))
        embeds2 = dy.reshape(dy.lookup_batch(self.embedding_matrix,x2),(len2[k],embedding_dim))

        if debug:
            print('embedding 1:', (len1[k],embedding_dim), embeds1.dim())
            print('embedding 2:', (embedding_dim,len2[k]), embeds2.dim())

        similarity = embeds1*dy.transpose(embeds2)
        if debug:
            print('similarity dimension:', (len1[k],len2[k]), similarity.dim())

        n_a = dy.softmax(similarity,d=0)
        n_b = dy.softmax(similarity,d=1)
        
        alpha = dy.transpose(n_a)*embeds1
        beta = n_b*embeds2
        
        if debug:
            print('alpha:',(len2[k],embedding_dim), alpha.dim())
            print('beta:',(len1[k],embedding_dim), beta.dim())
            
        v1i = w2*dy.transpose(dy.concatenate_cols([embeds1,beta])) + b2
        v2j = w3*dy.transpose(dy.concatenate_cols([embeds2,alpha])) + b3
        if debug:
            print('v1:', (embedding_dim,len1[k]), v1i.dim())
            print('v2:', (embedding_dim,len2[k]), v2j.dim())
        
        v1 = dy.mean_dim(v1i,[1],0)
        v1 = dy.reshape(v1,(embedding_dim,1)) 
        
        v2 = dy.mean_dim(v2j,[1],0)
        v2 = dy.reshape(v2,(embedding_dim,1)) 
        
        score = w*dy.concatenate([v1,v2],d=0) + b
        
        return score
        
    def train(self):
        len1 = self.len1
        len2 = self.len2
        embedding_dim = self.embedding_dim
        
        trainer = dy.AdamTrainer(self.pc)
        
        iter = 0
        for epochs in range(100):
            tl = 0
            for k in range(10000):
                iter += 1
                dy.renew_cg()
                x1 = self.data1[k,0:len1[k]]
                x2 = self.data2[k,0:len2[k]]
                label = self.labels[k]

                score = self.forward(x1,x2,label,k,tl)
                norm_score = dy.logistic(score)
                
                loss = dy.binary_log_loss(norm_score,dy.inputTensor([[label]]))
                loss.backward()
                trainer.update()
                tl += loss.scalar_value()
                #if iter % dev_iter == 0:
                    #predict_fn
                    
            print('Train loss after ' + str(epochs+1) + ' epochs: ' + str(tl/10000))

def main():
    model = decomposable_attention(embedding_dim=128,debug=False)
    model.train()
    
main()            

Train loss after 1 epochs: 0.701264912306
Train loss after 2 epochs: 0.0960322426866
Train loss after 3 epochs: 0.0179971929838
Train loss after 4 epochs: 0.0065578334452


KeyboardInterrupt: 

In [10]:
x1 = np.random.randint(5,size=10)
x2 = np.random.randint(5,size=15)

pc = dy.Model()
embedding_matrix = pc.add_lookup_parameters((168000,222))
a = dy.lookup_batch(embedding_matrix,x1)
b = dy.lookup_batch(embedding_matrix,x2)

c = a*b
print(c.dim())

ValueError: Mismatched input dimensions in MatrixMultiply: [{222X10} {222X15}]

In [28]:

pc = dy.Model()
s = pc.add_parameters((4))
s1 = pc.add_parameters((4))

dy.renew_cg()
sa = dy.parameter(s)
sb = dy.parameter(s1)
na = dy.concatenate([sa,sb])
na.npvalue()
#n_b = dy.softmax(similarity,d=1)

array([ 0.48167747, -0.07805443, -0.22833993,  0.49472263, -0.24113466,
        0.788912  ,  0.25677112, -0.74309444])